读取 路径规划_数据.json ，构建区域协同矩阵

In [1]:
import pandas as pd
from tqdm import tqdm

from itertools import permutations
from utils import load_json, save_json

In [2]:
df = pd.read_excel("data/CityList_333.xlsx")

In [3]:
def get_row(row):
    return (row["省份"], row["地级行政区"])

data = df.apply(get_row, axis=1).to_list()
# 自己到自己的位置的距离与费用是0
matrix = list(permutations(data, r=2))

In [4]:
matrix[0]

(('北京市', '北京市'), ('天津市', '天津市'))

In [ ]:
# 遍历 matrix 从路径规划_数据.json中，获取距离与时间

In [5]:
area_path_data = load_json("API_results/路径规划_数据.json")

In [6]:
head_key = next(iter(area_path_data.keys()))

In [7]:
matrix[0]

(('北京市', '北京市'), ('天津市', '天津市'))

In [8]:
matrix_data = []

for head_address, tail_address in matrix:
    key = str((head_address, tail_address))
    if key in area_path_data.keys():
        tmp_data = area_path_data[key]
    else:
        key = str((tail_address, head_address))
        tmp_data = area_path_data.get(key, {})

    if tmp_data == {}:
        matrix_data.append([head_address, tail_address, -1, -1])
    else:
        distance = tmp_data["result"]["routes"][0]["distance"]
        duration = tmp_data["result"]["routes"][0]["duration"]
        matrix_data.append([head_address, tail_address, distance, duration])

In [9]:
matrix_data[0]

[('北京市', '北京市'), ('天津市', '天津市'), 137490, 7274]

In [10]:
# 距离矩阵
distance_pd_data = {"row": [], "col": [], "value": []}
duration_pd_data = {"row": [], "col": [], "value": []}


def _set_v(_pd_data, head_address, tail_address, value):
    _pd_data["row"].append(head_address)
    _pd_data["col"].append(tail_address)
    _pd_data["value"].append(value)


for head_address, tail_address, distance, duration in matrix_data:
    _set_v(distance_pd_data, head_address, tail_address, distance)
    _set_v(duration_pd_data, head_address, tail_address, duration)

In [11]:
distance_df = pd.DataFrame(distance_pd_data)
distance_df = distance_df.pivot(index="row", columns="col", values="value")
distance_df.fillna(0, inplace=True)
distance_df.to_excel("output/全国两地之间距离.xlsx")

In [12]:
duration_df = pd.DataFrame(duration_pd_data)
duration_df = duration_df.pivot(index="row", columns="col", values="value")
duration_df.fillna(0, inplace=True)
duration_df.to_excel("output/全国两地通勤时间.xlsx")